# ラベルを付けて、学習に使うデータセットを作成する

In [1]:
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import numpy as np
import seaborn as sns
import pandas as pd
import sys
import stockpy as st
from datetime import datetime
from tqdm import tqdm

In [2]:
df = pd.read_csv('./valid_dataset_by_time.csv', header=0)

In [3]:
df

,0,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,61
0,AADI,0,0.209065,0.204498,0.200632,0.202389,0.189037,0.188335,0.187280,0.184469,...,0.151089,0.141251,0.147576,0.153549,0.157414,0.149332,0.161630,0.153900,0.147576,0.148630
1,AAL,1,0.782570,0.758311,0.747978,0.752920,0.744385,0.754268,0.752471,0.729110,...,0.572776,0.552111,0.571878,0.577269,0.573226,0.549865,0.550764,0.560647,0.541779,0.526505
2,AAME,2,0.412466,0.401318,0.416925,0.432532,0.416925,0.408007,0.408007,0.394629,...,0.430628,0.423864,0.412591,0.412591,0.408082,0.394554,0.408082,0.410336,0.419355,0.417100
3,AAOI,3,0.510982,0.550388,0.544574,0.513566,0.399871,0.433463,0.436047,0.454134,...,0.728036,0.666667,0.575581,0.589147,0.586563,0.489987,0.481912,0.503230,0.513566,0.487726
4,AAON,4,0.948376,0.956355,0.949597,0.948846,0.938802,0.966961,0.987986,0.999437,...,0.803923,0.781651,0.792787,0.789545,0.807448,0.813650,0.824786,0.832539,0.809421,0.791659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13400,ZTEK,2675,0.277193,0.291228,0.278947,0.285965,0.285965,0.280702,0.278947,0.282456,...,0.210526,0.215789,0.210526,0.201754,0.200000,0.210526,0.208772,0.201754,0.203509,0.208772
13401,ZUMZ,2676,0.343401,0.346330,0.345232,0.343767,0.343767,0.348343,0.342669,0.342486,...,0.307889,0.307706,0.312649,0.313381,0.302032,0.289401,0.308622,0.323815,0.313930,0.315211
13402,ZVRA,2677,0.452242,0.470760,0.475634,0.477583,0.468811,0.471735,0.469786,0.473684,...,0.458090,0.450292,0.462963,0.464912,0.444444,0.435672,0.434698,0.441520,0.425926,0.425926
13403,ZYME,2678,0.294618,0.294213,0.301902,0.301497,0.296236,0.296236,0.299069,0.293808,...,0.259004,0.254148,0.261028,0.262849,0.259814,0.263051,0.285310,0.293403,0.289761,0.291785


In [4]:
def process_data(data):
    try:
        data.pop()
        if ((0.02 < data[-1] - data[-2]) and (0.02 < data[-2] - data[-3])):        
            data.insert(0, 1)
        else:
            data.insert(0, 0)
        # リストをNumPy配列に変換
        data = np.array(data)
        return data
    except Exception as e:
        print("Error processing data")
        print(e)
        raise Exception

In [5]:
# これだめ
# df3 = pd.DataFrame()

# for index, row in tqdm(df.iterrows()):
#     row = process_data(row.tolist())
#     df_2 = pd.DataFrame(row)
#     df3 = pd.concat([df3, df_2], axis=0)

In [6]:
df3 = pd.DataFrame()

rows = []
for row in tqdm(df.itertuples(index=False)):
    row = process_data(list(row))
    rows.append(row)

df3 = pd.DataFrame(rows)

13405it [00:00, 21006.34it/s]


In [7]:
df3

,0,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,61
0,0,AADI,0,0.2090653549075961,0.2044975530177258,0.2006324744143179,0.2023893328944063,0.1890372553587384,0.1883345153176318,0.18728039687865,...,0.1700632574942183,0.1510892596296982,0.1412508822995628,0.1475755426695214,0.1535488413962489,0.1574139199996568,0.1493324011496098,0.1616303602462959,0.1539002197941243,0.1475755426695214
1,0,AAL,1,0.7825696270070966,0.7583108260199024,0.7479784118298114,0.7529200384794301,0.7443845249078221,0.7542677782070597,0.7524707919035536,...,0.575471711046796,0.5727762744340483,0.5521114032113547,0.5718777812822953,0.5772686545077905,0.5732255210099249,0.5498652131744836,0.5507637063262366,0.5606468739404511,0.5417789890212636
2,0,AAME,2,0.4124658193282393,0.4013180690574547,0.4169249086858012,0.432531721437268,0.4169249086858012,0.4080067299706773,0.4080067299706773,...,0.4351370317711595,0.4306278439098334,0.4238640621178444,0.4125910924645292,0.4125910924645292,0.4080818777263235,0.3945543141423453,0.4080818777263235,0.4103364985338662,0.4193548742565183
3,0,AAOI,3,0.5109819173980018,0.5503876427461977,0.5445736792288003,0.513565894338304,0.3998708165547023,0.4334625475820658,0.4360465245223681,...,0.7086563689551647,0.7280362268108662,0.6666666666666666,0.5755814025124267,0.5891472968507308,0.5865633199104285,0.4899870970571877,0.4819121614178845,0.5032299865770952,0.513565894338304
4,0,AAON,4,0.9483763191002468,0.9563545350849854,0.9495965560623923,0.9488455500068078,0.9388024168994538,0.966960895847086,0.9879858389704496,...,0.801667990579676,0.8039234285711827,0.7816509851267891,0.7927872337359276,0.789544991210121,0.8074475537937797,0.8136500486008358,0.824786243436091,0.8325393619449111,0.8094210553146127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13400,0,ZTEK,2675,0.2771929992606514,0.2912280740646922,0.2789473836111565,0.2859649210131769,0.2859649210131769,0.2807017679616616,0.2789473836111565,...,0.2157894842512393,0.210526331199724,0.2157894842512393,0.210526331199724,0.2017543885332873,0.2000000041827822,0.210526331199724,0.2087719468492189,0.2017543885332873,0.2035087728837924
13401,0,ZUMZ,2676,0.343401059163279,0.3463298500159941,0.3452315621747112,0.3437671667483536,0.3437671667483536,0.3483434068199635,0.3426688439931298,...,0.3088046953943694,0.3078894264316828,0.3077063726391455,0.31264872029583,0.3133809354659793,0.3020318447262527,0.2894014298096764,0.3086216416018321,0.3238147572430193,0.3139300619296502
13402,0,ZVRA,2677,0.4522416922978252,0.4707602159802721,0.4756335288086786,0.4775828446449701,0.4688109001439806,0.4717348971360956,0.4697855812998041,...,0.4473684259447743,0.4580896398066997,0.4502923764615337,0.4629629526351062,0.4649122684713976,0.4444444289526593,0.4356724844516698,0.4346978497712018,0.4415204784358998,0.4259259052702123
13403,0,ZYME,2678,0.2946175831460462,0.2942128794126447,0.3019020766712496,0.301497372937848,0.2962363594849803,0.2962363594849803,0.2990692084294469,...,0.2533387479085547,0.259004465094824,0.2541481360780218,0.2610279258698235,0.2628490540754584,0.2598138532642911,0.2630514059421592,0.2853096095485071,0.2934034912431776,0.2897612348319078


In [8]:
df3[0].value_counts()

0    13329
1       76
Name: 0, dtype: int64

In [9]:
df3.to_csv('labeled_valid_dataset.csv', index=False)